In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_csv('D:/train.csv')
df.shape

(150096, 3)

In [3]:
df.head().T

,0,1,2,3,4
number,0,1,2,3,4
Text,\nخبرنامه دانشگاه علم و صنعت ايران \nشماره ياز...,\nتا پايان سال 1378 دهها زمين فوتبال و \nسالن ...,\nانجمن توليدكنندگان تجهيزات صنعت نفت تشكيل شد...,\nكرتين براي سومين بار نخست وزير كانادا \nشد \...,\nخداحافظ رفقا \nنمايندگان اروپاي شرقي در جام ...
Category,Science and Culture,Sport,Economy,Miscellaneous.World News,Sport


In [4]:
pd.DataFrame(df.Category.unique()).values

array([['Science and Culture'],
       ['Sport'],
       ['Economy'],
       ['Miscellaneous.World News'],
       ['Miscellaneous.Urban'],
       ['Social.Women'],
       ['Social'],
       ['Literature and Art'],
       ['Politics'],
       ['Miscellaneous'],
       ['Economy.Bank and Bourse'],
       ['Politics.Iran Politics'],
       ['Tourism'],
       ['Social.Religion'],
       ['Miscellaneous.Picture'],
       ['Miscellaneous.Happenings'],
       ['Science and Culture.Science.Book'],
       ['Literature and Art.Art'],
       ['Miscellaneous.Islamic Councils'],
       ['Literature and Art.Art.Cinema'],
       ['Science and Culture.Science.Information and Communication Technology'],
       ['Economy.Oil'],
       ['Economy.Commerce'],
       ['Natural Environment'],
       ['Science and Culture.Science'],
       ['Economy.Industry'],
       ['Economy.Agriculture'],
       ['Sport.World Cup'],
       ['Miscellaneous.Picture.Caricature'],
       ['Literature and Art.Art.Music'],
   

In [5]:
pd.DataFrame(df.Category.unique()).shape

(34, 1)

In [6]:
df['category_id'] = df['Category'].factorize()[0]
category_id_df = df[['Category', 'category_id']].drop_duplicates()

category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)

df.head()

,number,Text,Category,category_id
0,0,\nخبرنامه دانشگاه علم و صنعت ايران \nشماره ياز...,Science and Culture,0
1,1,\nتا پايان سال 1378 دهها زمين فوتبال و \nسالن ...,Sport,1
2,2,\nانجمن توليدكنندگان تجهيزات صنعت نفت تشكيل شد...,Economy,2
3,3,\nكرتين براي سومين بار نخست وزير كانادا \nشد \...,Miscellaneous.World News,3
4,4,\nخداحافظ رفقا \nنمايندگان اروپاي شرقي در جام ...,Sport,1


In [7]:
df['Text'].dropna(inplace=True)
df['Text']= [word_tokenize(entry) for entry in df['Text']]
df['Text'] = df["Text"].map(' '.join)
df['Text'] = [(t.replace('\n',' ')
            .replace('  ',' ')
            .replace(':',' ')
            .strip()) for t in df['Text']]

print(df['Text'])

0         خبرنامه دانشگاه علم و صنعت ايران شماره يازدهم ...
1         تا پايان سال 1378 دهها زمين فوتبال و سالن ورزش...
2         انجمن توليدكنندگان تجهيزات صنعت نفت تشكيل شد ن...
3         كرتين براي سومين بار نخست وزير كانادا شد ژان ك...
4         خداحافظ رفقا نمايندگان اروپاي شرقي در جام 2002...
                                ...                        
150091    مشكل بزرگ سدسازي وزير نيرو   در كشور كمبود اعت...
150092    خودسوزي يك هوادار پ ك ك در شهر دياربكر تركيه آ...
150093    از نيمه دوم سال جاري در 600 دستگاه اتوبوس كارت...
150094    در كتابخانه مركز دانشگاه تهران و دانشكده هنرها...
150095    پيام رئيس جمهوري به سران كشورهاي عضو سازمان اس...
Name: Text, Length: 150096, dtype: object


In [8]:
from __future__ import unicode_literals
from hazm import *

normalizer = Normalizer()
df['Text'] = df['Text'].map(lambda x: normalizer.normalize(x))

stemmer = Stemmer()
df['Text'] = df['Text'].map(lambda x: stemmer.stem(x))

lemmatizer = Lemmatizer()
df['Text'] = df['Text'].map(lambda x: lemmatizer.lemmatize(x))

In [9]:
print(df.Text)

0         خبرنامه دانشگاه علم و صنعت ایران شماره یازدهم ...
1         تا پایان سال ۱۳۷۸ دهها زمین فوتبال و سالن ورزش...
2         انجمن تولیدکنندگان تجهیزات صنعت نفت تشکیل شد ن...
3         کرتین برای سومین بار نخست وزیر کانادا شد ژان ک...
4         خداحافظ رفقا نمایندگان اروپای شرقی در جام ۲۰۰۲...
                                ...                        
150091    مشکل بزرگ سدسازی وزیر نیرو در کشور کمبود اعتبا...
150092    خودسوزی یک هوادار پ ک ک در شهر دیاربکر ترکیه آ...
150093    از نیمه دوم سال جاری در ۶۰۰ دستگاه اتوبوس کارت...
150094    در کتابخانه مرکز دانشگاه تهران و دانشکده هنرها...
150095    پیام رئیس جمهوری به سران کشورهای عضو سازمان اس...
Name: Text, Length: 150096, dtype: object


In [10]:
tfidf = TfidfVectorizer(max_features=5000)
features = tfidf.fit_transform(df.Text).toarray()
labels = df.category_id

In [11]:
N = 5
for Category, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  words = [v for v in feature_names if len(v.split(' ')) == 1]
  print("\n%s:" %(Category))
  print("Most occurred words: %s" %(', '.join(words[-N:])))


Economy:
Most occurred words: پارس, اقتصادی, گذاری, سرمایه, قیمت

Economy.Agriculture:
Most occurred words: هکتار, گندم, آبیاری, کشاورزان, کشاورزی

Economy.Bank and Bourse:
Most occurred words: بهادار, اوراق, بانک, سهام, بورس

Economy.Commerce:
Most occurred words: واردات, صادرات, گمرکی, کالا, گمرک

Economy.Dwelling and Construction:
Most occurred words: پروانه, تراکم, مقاوم, ساختمان, مسکن

Economy.Industry:
Most occurred words: تولید, صنایع, صنعت, خودرو, خودروسازی

Economy.Oil:
Most occurred words: نفتی, انرژی, بشکه, گاز, نفت

Literature and Art:
Most occurred words: نمایش, سینمای, موسیقی, جشنواره, فیلم

Literature and Art.Art:
Most occurred words: فیلمی, او, فیلمنامه, آدم, فیلم

Literature and Art.Art.Cinema:
Most occurred words: فیلمی, سینما, فیلمنامه, سینمای, فیلم

Literature and Art.Art.Music:
Most occurred words: کنسرت, استاد, پاپ, ارکستر, موسیقی

Literature and Art.Art.Theater:
Most occurred words: قصه, متن, نمایش, نمایشنامه, تئاتر

Literature and Art.Literature:
Most occurred 

In [ ]:
X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(features, 
                                                               labels, 
                                                               df.index, test_size=0.25, 
                                                               random_state=1)
model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred, 
                                    target_names= df['Category'].unique()))

In [ ]:
test_df = pd.read_csv('D:/test.csv')
test_df.shape

In [ ]:
test_pd = pd.read_csv('D:/test.csv')

test_pd['Text'].dropna(inplace=True)
test_pd['Text']= [word_tokenize(entry) for entry in test_pd['Text']]
test_pd['Text'] = test_pd["Text"].map(' '.join)
test_pd['Text'] = [(t.replace('\n',' ')
            .replace('  ',' ')
            .replace(':',' ')
            .strip()) for t in test_pd['Text']]


test_pd['Text'] = test_pd['Text'].map(lambda x: normalizer.normalize(x))
test_pd['Text'] = test_pd['Text'].map(lambda x: stemmer.stem(x))
test_pd['Text'] = test_pd['Text'].map(lambda x: lemmatizer.lemmatize(x))

In [ ]:
test_pd.head

In [ ]:
tfidf_test = TfidfVectorizer(max_features=5000)
features_test = tfidf_test.fit_transform(test_pd.Text).toarray()

In [ ]:
y_pred = model.predict(features_test)

In [ ]:
id_to_category

In [ ]:
y_pred_final = []
for i in y_pred:
    y_pred_final.append(id_to_category[i])

In [ ]:
out = pd.DataFrame(data=y_pred_final, columns=['Category'])
out.to_csv('D:/test_labels.csv')